In [137]:
pip install keras-tuner

In [138]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [139]:
fashion_mnist=keras.datasets.fashion_mnist

In [140]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [141]:
train_images=train_images/255.0
test_images=test_images/255.0

In [142]:
print(train_images[0].shape)
print(test_images[0].shape)

(28, 28)
(28, 28)


In [143]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28)
(10000, 28, 28)


In [144]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [145]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [146]:
def build_model(hp):
  model=keras.Sequential([
    keras.layers.Conv2D(
      filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
      kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
      activation='relu', 
      input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
      filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
      kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
      activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
      units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
      activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')#output layer
])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [147]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters


In [151]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='output',project_name="Mnist Fashion")
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)
model=tuner_search.get_best_models(num_models=1)[0]

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [152]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        73792     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2949200   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 3,025,082
Trainable params: 3,025,082
Non-trainable params: 0
_________________________________________________________________
